In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Exploring the database

Import relevant packages and increase the display length when printing using `pandas` to `160`:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.display.width = 160

## Reading the data

Read the `pickle` file which contains the database and show the first elements of the dataframe:

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/ALCE/SPE_Database.pkl')
df.head()

If your data is in another format, you can use other reading methods from the `pandas` library. For instance, if your data was in an Excel spreadsheet, you can use the `pd.read_excel` method to read your data and use it as an input to the neural network.

## Viewing the data

The `df.describe()` method gives some insight in the range of values and the distribution of numerical data. You can also explore the dataset by printing `df.columns`, which will show the name of all of the dataframe's columns and exploring the values of each column.

In [ ]:
df.describe()

The `Series.value_counts()` method counts the number of times a given value is present in a dataframe. For our database, the following cell counts the number of examples of a given study. It is seen that the data from Shoham is very significant in this database, with more than 5000 examples, which represents roughly $47\%$ of the data.

In [ ]:
df['author'].value_counts()

In [ ]:
df['author'].value_counts()/len(df)

Similarly, the number of points of each flow pattern can be computed usind the `value_counts` method.

In [ ]:
df['flow_pattern_code'].value_counts()

In [ ]:
df['flow_pattern_name'].value_counts()

The cell below plots values of the variables selected in `columns` groupped by the `author` field. It is possible to check which variables were studied for each author. This is done by using a `boxplot`.

In [ ]:
columns = ['v_sl', 'v_sg', 'rho_l', 'rho_g', 'mu_l', 'mu_g', 'D']
figsize = (8, 8)

for c in columns:
    print('-'*120 + f'\n{c}')
    print(df.groupby('author')[c].describe(percentiles=[]))
    
    fig, ax = plt.subplots(figsize=figsize)
    df.boxplot(by='author', column=c, rot=90, ax=ax)
    ax.set_yscale('log')
    ax.set_ylabel(c)
    plt.show()

Histograms are easy to plot as well using the `pandas` library. The cells below show the liquids and gases present in the database and their frequency.

In [ ]:
df['liquid'].hist(xrot=90, bins=len(df.liquid.unique()))

In [ ]:
df['gas'].hist(xrot=90, bins=len(df.gas.unique()))

As a last example of exploring the dataset, the points are grouped by the `author` and `flow_pattern_name` fields and the number of points in this subgroup of the dataframe is printed. For instance, it can be seen that `Abdul-Majeed (1995)` has 22 points with an `Annular` flow pattern. Also, it only has 4 different flow patterns as it does not have any points with `Dispersed Bubble` or `Bubble Flow`.

In [ ]:
for i, df_sub in df.groupby(['author', 'flow_pattern_name']):
        print(f'{i[0][0:20]:20} - {i[1][0:20]:20}: {len(df_sub)}')

It can be seen that the dataset covers a wide range of values for some variables (e.g. `v_sl`, `v_sg`), while other variables are underrepresented. This is not a problem if the `model` is being used to `predict` values in a range similar to what is present in the database.

# Data Splitting

There are many strategies in splitting the database into training, validation and testing datasets. In here, we will present the most simple one of randomly sampling points from the original dataset. A seed is given to ensure repeatibility between different runs of the split and make sure that the points in the test dataset are not involved in the improovement of the `model`.

In [ ]:
import numpy as np
import random
random.seed(42)

It is easy to sample a given frequency using the `.sample` method. Here, `df` is sampled and $60\%$ of the points fo to `df_train`. Then, a dataframe with the remaining points is obtained by dropping the points in `df_train` from `df` using the command `df.drop(df_train.index)`. $50\%$ of these points are randomly sampled and go to `df_val`, while the remaining points which are not in `df_train` or `df_val` go to `df_test`.

In [ ]:
df_train = df.sample(frac=0.6, random_state=42)
df_val = df.drop(df_train.index).sample(frac=0.5, random_state=42)
df_test = df.drop(df_train.index).drop(df_val.index)

As a check of the code above, we can check the lengths of each dataframe.

In [ ]:
N_points = [len(df_train), len(df_val), len(df_test)]
print(N_points)
print(sum(N_points), len(df))

To check if there is an intersection of the data in `df_train`, `df_val` and `df_test`, the `merge` method can be used. The cell below shows that there is no intersection between `df_train` and `df_val`.

In [ ]:
pd.merge(df_train, df_val, how='inner', on=df.columns.to_list())

# Neural Networks

The neural networks can be implemented very easily by using the `keras` API for tensorflow. First, let's import all relevant objects.

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Input, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import Callback

It is good to have some kind of feedback during training to make sure the code is running (at least in the first few iterations). Since our dataset is very small, printing at every `epoch` will be too frequent, so the following `class` is created and will be used as a `Callback` in the `model.fit` method. At the end of each epoch (hence the use of `on_epoch_end`), by using this callback, the code will print the desired information only if `epoch` is divisible by `N`. 

In [ ]:
class PrintEveryNEpochs(Callback):
    def __init__(self, N):
        super(PrintEveryNEpochs, self).__init__()
        self.N = N
    def on_epoch_end(self, epoch, logs=None):
        if (epoch % self.N) == 0:
            print(
            "Epoch {:6}: Train loss: {:12.8f} Validation loss: {:12.8f} Accuracy: {:6.3f} Validation Accuracy: {:6.3f}"
            .format(epoch, logs["loss"], logs["val_loss"], logs['accuracy'], logs['val_accuracy'])
            )

## Selection of inputs and outputs and preprocessing

The `inputs` and `output` variables are lists of the desired inputs and outputs for the neural network. Specifically, these lists contain the name of the columns of `df` which should be used to get input and output values, respectively.

In [ ]:
df.columns

In [ ]:
inputs = ['rho_l', 'rho_g', 'mu_l', 'mu_g', 'sigma', 'D', 'sin_angle', 'cos_angle', 'v_sl', 'v_sg']
output = ['flow_pattern_code']

It is a good practice to normalize the data so that it is in a nice numerical range and the values are more significant at first sight. This also gives some intuition for the neural network, as a negative value represents a low value in comparison with the dataset, while a given value (e.g. 800 kg/m$^3$) has no immediately obvius significance for the neural network. This is done by using the `scaler` class from `sklearn`. Notice that the normalization is fitted using `df_train` and the transformation is performed on `df_val` and `df_test` using the values obtained from `df_train`. This is important, as it prevent that data coming from `df_test` and `df_val` to be implictly transfered to `df_train`, contaminating the data.

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(df_train[inputs].values)
X_val, X_test = scaler.transform(df_val[inputs].values), scaler.transform(df_test[inputs].values)

We can check that scaling was successful by evaluating the mean and standard deviation of the resulting `X`s. Notice that the `mean` values for the training dataset are on the order of the machine precision, while the values for validation and training are not exactly zero.

In [ ]:
X_train.mean(axis=0), X_val.mean(axis=0), X_test.mean(axis=0)

Similarly, the standard deviation for each feature is `1` for the training values, while it is only close to 1 for validaiton and test datasets. This is expected, as the values are slightly different in each dataset.

In [ ]:
X_train.std(axis=0), X_val.std(axis=0), X_test.std(axis=0)

We can save the scaler for later use by using the cell below.

In [ ]:
import pickle
if not os.path.exists('Models'):
    os.makedirs('Models')
with open('Models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

`df[output].values` are numerical data with each value corresponding to a different flow pattern. In the neural network model, the `Y` value corresponds to the probability of that point being a given flow pattern. Since each point only has one flow pattern, the probability is `1` for that given flow pattern. Hence, we want to transform each element of the `df[output]` column from a numerical value to an array with a value of `1` for the corresponding flow pattern and `0` for the other ones. This is also known as one-hot encoding. This is done by using the 'to_categorical' function.

In [ ]:
Y_train = to_categorical(df_train[output].values)
Y_val = to_categorical(df_val[output].values)
Y_test = to_categorical(df_test[output].values)

Notice that the `Y` values now have 6 columns, corresponding to the six different flow patterns of this problem.

In [ ]:
Y_train.shape, Y_val.shape, Y_test.shape

In [ ]:
Y_train

## Building the neural networks

With the data already split into training, validation and test and the normalizing and one-hot encoding steps completed, the model can be built. In this example, we will use batches of `64` examples, two hidden layers with `16` neurons each and the `tanh` activation function. `fname` is the name of the file where the model will be saved after training. This is useful because there is no need to train the model every time and it can be readily loaded from the file. Dropout rates of $20\%$ are used in the intermediate layers, while the last layer has a dropout rate of $50\%$. The number of epochs was selected to be high enough to obtain the learning curve.

In [ ]:
n_inputs = len(inputs)
n_flow_patterns = len(df_train.flow_pattern_code.unique())
n_batch= 64
n_hidden_layers = [16, 16]
fname = "Models/trained_model_16_16.hdf5"
fname = "Models/trained_model_16_16_dropout.hdf5"
activation_layer = 'tanh'
dropout_rate_mid = 0.2
dropout_rate_final = 0.5
n_epochs = int(50e3)

The last activation function is `'softmax'`. This is because we want to interpret the values given by the `model` as a probability for each flow pattern and this activation funtion already normalizes the outputs so they can be interpreted that way. The model is plotted to present the dataflow. A summary of the parameters and shapes is also given.

In [ ]:
main_input = Input(shape=(n_inputs,), name='main_input')
x = main_input
while len(n_hidden_layers) > 1:
    x = Dense(units=n_hidden_layers.pop(0), activation=activation_layer)(x)
    x = Dropout(dropout_rate_mid)(x)
x = Dense(units=n_hidden_layers.pop(0), activation='relu')(x)
x = Dropout(dropout_rate_final)(x)
flow_pattern = Dense(units=n_flow_patterns, activation='softmax', name='flow_pattern')(x)

#%% Model build
model = Model(inputs=[main_input], outputs=[flow_pattern])

plot_model(model, to_file='model.png', show_shapes=True, dpi=300)

In [ ]:
model.summary()

In total, this model has 550 trainable parameters. A rule of thumb in order to ensure generalization is to have 10 times more examples than the number of parameters, so this is consistent with the data budget we have. This can also be evaluated by looking at the learning curves and comparing the validation and training errors. 

## Training the network
Now we set the optimizer options and compile the model. The categorical crossentropy is used as a loss function and the accuracy is selected as a metric.

In [ ]:
adam = optimizers.Adam(1e-3)
model.compile(loss={'flow_pattern':'categorical_crossentropy',
                    },
              optimizer=adam,
              metrics={'flow_pattern':['accuracy']
                       },
             )

In [ ]:
X_train.shape, Y_train.shape

The model is fitted to the data using the `model.fit` method. After training the model is saved and the history during training is kept in `history`.

In [ ]:
history = model.fit(x=X_train, y=Y_train, verbose=0, batch_size=n_batch, epochs=n_epochs,
                    validation_data=(X_val, Y_val), callbacks=[PrintEveryNEpochs(10)])
model.save(fname)

## Optimizing hyperparameters

The `history` object is used to plot the learning curve of our model. It is seen that the validation error is still decreasing with the training error, however the training error is stagnant (although it is still decreasing slowly). This indicates that either too much regularization is being used or the model is too simple. We can improve this by either decreasing the regularization (in this case, the dropout rate) or increasing the model complexity (by increasing the number of layers or neurons). 

In [ ]:
fig, ax = plt.subplots()
ax.loglog(history.history['loss'], label='Training Error')
ax.loglog(history.history['val_loss'], label='Validation Error')
ax.set_xlabel("Epochs")
ax.set_ylabel("Flow pattern categorical crossentropy loss")
ax.legend()
plt.tight_layout()

fig, ax = plt.subplots()
ax.plot(history.history['accuracy'], label='Training Error')
ax.plot(history.history['val_accuracy'], label='Validation Error')
ax.set_xlabel("Epochs")
ax.set_ylabel("Flow pattern accuracy")
ax.set_ylim([0, 1])
ax.legend()
plt.tight_layout()

The curve below is for the same model but with no dropout. It is seen that after $\approx 3x10^3$, the model starts overfitting and further training will actually result in worse results. However, it is seen that the model accuracy is higher than the previous one with dropout rate.

In [ ]:
fig, ax = plt.subplots()
ax.loglog(history.history['loss'], label='Training Error')
ax.loglog(history.history['val_loss'], label='Validation Error')
ax.set_xlabel("Epochs")
ax.set_ylabel("Flow pattern categorical crossentropy loss")
ax.legend()
plt.tight_layout()

fig, ax = plt.subplots()
ax.plot(history.history['accuracy'], label='Training Error')
ax.plot(history.history['val_accuracy'], label='Validation Error')
ax.set_xlabel("Epochs")
ax.set_ylabel("Flow pattern accuracy")
ax.set_ylim([0, 1])
ax.legend()
plt.tight_layout()

This is an iterative process and the hyperparameter tuning (i.e. dropout rates, hidden layer units, etc.) can be tuned using different approaches. Let's assume that we already did this hyperparameter optimization and let's explore the `model` capabilities. First, the model is loaded from `fname` using `load_model`.

In [ ]:
model = load_model(fname)

## Using the model

The two functions below take a model, some input parameters and create a flow pattern map and flow probability as a function of the superficial velocities.

In [ ]:
from matplotlib.colors import ListedColormap
import seaborn as sns

fp_dict = {
    1:'Dispersed Bubble',
    2:'Stratified Smoth',
    3:'Stratified Wavy',
    4:'Annular',
    5:'Intermitent',
    0:'Bubble Flow',
}

def plot_flow_pattern(model, rho_l=1000., rho_g=1.8, mu_l=1e-3, mu_g=0.000015, sigma=0.070, D=0.051, angle=0):
        sin_angle = np.sin(angle*np.pi/180)
        cos_angle = np.cos(angle*np.pi/180)
        vsls = np.logspace(-2, 1, 300)
        vsgs = np.logspace(-2, 2, 300)
        vsls, vsgs = np.meshgrid(vsls, vsgs)
        X = np.array([
                [rho_l, rho_g, mu_l, mu_g, sigma, D, sin_angle, cos_angle, vsl, vsg]
                for (vsl, vsg) in zip(vsls.flatten(), vsgs.flatten())]
            )
        X = scaler.transform(X)
        flow_patterns = model.predict(X).argmax(axis=1)
        
        fig, ax = plt.subplots()
        cmap = ListedColormap(sns.color_palette('Paired', n_colors=6).as_hex())
        ax.contourf(vsgs, vsls, flow_patterns.reshape(vsgs.shape), cmap=cmap, vmin=0, vmax=5)
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_xlabel('$v_{sg}$ [m/s]')
        ax.set_ylabel('$v_{sl}$ [m/s]')
        plt.tight_layout()

def plot_flow_probability(model, cmap='RdBu', rho_l=1000., rho_g=1.8, mu_l=1e-3, mu_g=0.000015, sigma=0.070, D=0.051, angle=0):
        sin_angle = np.sin(angle*np.pi/180)
        cos_angle = np.cos(angle*np.pi/180)
        vsls = np.logspace(-2, 1, 300)
        vsgs = np.logspace(-2, 2, 300)
        vsls, vsgs = np.meshgrid(vsls, vsgs)
        X = np.array([
                [rho_l, rho_g, mu_l, mu_g, sigma, D, sin_angle, cos_angle, vsl, vsg]
                for (vsl, vsg) in zip(vsls.flatten(), vsgs.flatten())]
            )
        X = scaler.transform(X)
        flow_patterns = model.predict(X)
        fig, axes = plt.subplots(3,2, sharex=True, sharey=True, figsize=(10,10))
        for pattern_code, ax in enumerate(axes.flatten()):
            im = ax.pcolormesh(vsgs, vsls, flow_patterns[:, pattern_code].reshape(vsgs.shape),
                               vmin=0, vmax=1,
                               cmap=cmap)
            plt.colorbar(im, ax=ax)
            ax.set_xscale('log')
            ax.set_yscale('log')
            ax.set_xlabel('$v_{sg}$ [m/s]')
            ax.set_ylabel('$v_{sl}$ [m/s]')
            ax.set_title(fp_dict[pattern_code])
            plt.tight_layout()

We can use this functions to plot the flow pattern map for different conditions. This is done below.

In [ ]:
plot_flow_pattern(model, angle=0)
plot_flow_probability(model, angle=0)

Notice that the probability of being stratified smooth is actually not that high in the stratified region. However, it is higher than other flow patterns, so the predicted flow pattern is stratified smooth, although with a lower confidence. This already illustrates one of the benefits of using this type of model. It also gives some kind of confidence values. Let's check the flow pattern for vertical case.

In [ ]:
plot_flow_pattern(model, angle=90)
plot_flow_probability(model, angle=90)

It is seen that in the vertical case the model predicts intermittent flow, even though it should predict bubbly flow in the region of low `v_sl` and `v_sg`. Can you think of a reason why this is the case?

Let's check the number of points for each flow pattern.

In [ ]:
df_train.flow_pattern_code.value_counts()

In [ ]:
df_train.flow_pattern_code.value_counts()/len(df_train)

In [ ]:
df_val.flow_pattern_code.value_counts()

In [ ]:
df_val.flow_pattern_code.value_counts()/len(df_val)

The number of points with intermittent flow pattern represents almost $52\%$ of the dataset, while points with bubbly flow pattern are only $3.7\%$. When we were using the categorical crossentropy loss, it is implicit that the number of points in each class is balanced, i.e. they have the same fraction of points. However, in our data the number of points with bubbly flow are underrepresented and it was easier for the model to just predict intermittent flow than to try and predict bubbly flow. This is more clearly seen if we check the confusion matrix of our model.

## Evaluating the model

The confusion matrix shows how a given class (in our case, a flow pattern) is predicted by the model. The rows indicate the real value while the columns represent the predicted value. Hence, we see that $99.6\%$ of the points which are Bubble flow are predicted as Intermittent flow. Notice that no points are predicted as bubble flow.

In [ ]:
from sklearn.metrics import confusion_matrix
def calculate_confusion_matrix(df_):
    X = scaler.transform(df_[inputs].values)
    predicted_fp = model.predict(X).argmax(axis=1)

    fps = [fp_dict[c] for c in range(6)]
    df_results = pd.DataFrame(confusion_matrix(df_['flow_pattern_code'], predicted_fp),
                              columns=fps, index=fps)
    return (df_results.T/df_results.sum(axis=1)).T

In [ ]:
calculate_confusion_matrix(df_train)

In [ ]:
calculate_confusion_matrix(df_val)

In [ ]:
calculate_confusion_matrix(df_test)

As a comparison with the Barnea model, the confusion matrix for the Barnea flow pattern is given below:

In [ ]:
fps = [fp_dict[c] for c in range(6)]
df_results_barnea = pd.DataFrame(confusion_matrix(df['flow_pattern_code'], df['barnea_flow_pattern_code']),
                          columns=fps, index=fps)
(df_results_barnea.T/df_results_barnea.sum(axis=1)).T

It is seen that Barnea is a better predictor for bubble flow. However, it gives worse results for the other flow patterns.

# Example

A given line transport 500x10$^3$ BPD (@ P,T) and 5.0  MM ft$^3$/day (@ P,T), The fluid properties are presented below.  To solve Flow oscillation problems caused by slugs, a finger type slug catcher of one bottle is proposed at the end of the line. 

![image.png](attachment:image.png)

The idea is to stratify the phases and extract the gas at the top of the line.  The current line is horizontal and 26 in. ID. For this condition answer the following questions:
 
|     Liquid Density     |     Liquid Viscosity    |     Surface Tension    |     Gas Density     |     Gas Viscosity    |
|:----------------------:|:-----------------------:|:----------------------:|:-------------------:|:--------------------:|
|         51         |          3         |         32        |        5           |       1E-02     |
|        [Lb/ft$^3$]     |           [cp]          |        [Dyna/cm]       |       [Lb/ft$^3$]   |          [cp]        |


Part 1:
 What is the pipe diameter that will stratify the phases in horizontal flow, consider: 26, 30, 36, 48 and  60 in.
 
Part 2:
 What is the inclination angle that can stratified the phases for the 26 in. ID line.


## Problem inputs and preprocessing

The input data is given below:

In [ ]:
q_l_sc_field = 500e3 # stb/Day
q_g_sc_field = 5e6 # ft^3/Day
rho_l_field = 51 # lb/ft3
mu_l_field = 3 # cp
sigma_field = 32 # Dyne/cm
rho_g_field = 5 # lb/ft3
mu_g_field = 1e-2 # cp

Conversion coefficients:

In [ ]:
lb_to_kg = 0.453592
ft_to_m = 12*0.0254
stb_to_ft3 = 42*231/12**3
days_to_sec = 24*3600
dyne_cm_to_N_m = 1e-3
cp_to_Pa_s = 1e-3

Convert to SI units:

In [ ]:
rho_l = rho_l_field*lb_to_kg/ft_to_m**3
rho_g = rho_g_field*lb_to_kg/ft_to_m**3
sigma = dyne_cm_to_N_m*sigma_field
mu_l = cp_to_Pa_s*mu_l_field
mu_g = cp_to_Pa_s*mu_g_field
q_l_sc = q_l_sc_field*stb_to_ft3*ft_to_m**3/days_to_sec
q_g_sc = q_g_sc_field*ft_to_m**3/days_to_sec

## Part 1 - Selection of $D$
Create a grid of values for $D$ and $\theta$:

In [ ]:
Ds = np.linspace(26, 60, 100)*0.0254
angles = np.linspace(-45, 45, 91)
Ds, angles = np.meshgrid(Ds, angles)

Create the array with corresponding inputs, calculating $\sin \theta$, $\cos \theta$, $v_{sl}=q_{l,sc}/A$ and $v_{sg}=q_{g,sc}/A$. Normalize using `scaler.transform` (this is the same `scaler` which was used in training). Predict using the `model.predict` method. The probabilities for stratified smooth and stratified wavy are given by the columns 3 and 4 of `fp` (remember that python indexing starts at 0).

In [ ]:
X = np.array([
                [rho_l, rho_g, mu_l, mu_g, sigma, D, np.sin(angle*np.pi/180), np.cos(angle*np.pi/180),
                 q_l_sc/(np.pi*D**2/4), q_g_sc/(np.pi*D**2/4)]
                for (D, angle) in zip(Ds.flatten(), angles.flatten())]
            )
X = scaler.transform(X)
fp = model.predict(X)
stratified = fp[:, 2] + fp[:, 3]

Plot the probability of being stratified as a function of $D$ and $\theta$.

In [ ]:
fig, ax = plt.subplots(1, sharex=True, sharey=True, figsize=(6, 4.5))
im = ax.pcolormesh(Ds/0.0254, angles, stratified.reshape(Ds.shape),
                   vmin=0, vmax=1,
                   cmap='RdBu')
plt.colorbar(im, ax=ax)
ax.set_xlabel('$D$ [in]')
ax.set_ylabel('$\\theta$ [°]')
ax.set_title('Probability of Stratified flow')
ax.set_xticks([26, 30, 36, 48, 60])
plt.tight_layout()

We can also use slices of the previous plot to get probability of stratified flow for constant $\theta$ or $D$. Probability of stratified flow for a constant angle is given below. You can change the index `i` and check how the probability curve changes for different inclinations.

In [ ]:
i = 45
fig, ax = plt.subplots()
stratified = stratified.reshape(Ds.shape)
ax.plot(Ds[i]/0.0254, stratified[i])
ax.set_xticks([26, 30, 36, 48, 60])
ax.set_title(f'Probability of Stratified flow for {angles[i].mean()}°')
ax.set_ylim([0, 1])
ax.set_xlabel('D [in]')

## Part 2 - Selection of $\theta$

Similarly, we can slice the data in a different axis and get the behavior of the stratified probability for a constant diameter. The index `i` can also be changed to show different diameters.

In [ ]:
i = 0
fig, ax = plt.subplots()
ax.plot(angles.T[i], stratified.T[i])
ax.set_title(f'Probability of Stratified flow for {Ds.T[i].mean()/0.0254:.3} in')
ax.set_ylim([0, 1])
ax.set_xlabel('$\\theta$ [°]')

In [ ]:
i = 64
fig, ax = plt.subplots()
ax.plot(angles.T[i], stratified.T[i])
ax.set_title(f'Probability of Stratified flow for {Ds.T[i].mean()/0.0254:.3} in')
ax.set_ylim([0, 1])
ax.set_xlabel('$\\theta$ [°]')

In [ ]:
i = 99
fig, ax = plt.subplots()
ax.plot(angles.T[i], stratified.T[i])
ax.set_title(f'Probability of Stratified flow for {Ds.T[i].mean()/0.0254:.3} in')
ax.set_ylim([0, 1])
ax.set_xlabel('$\\theta$ [°]')

 - Try to answer the questions of the example based on the plots above.
 - Is it risky to select a 26 in diameter with a downward inclination? 
 - How would your answer change if the flow conditions of interest were significantly different than the ones present in the database?
 - Try to plot an operation envelope ($v_{sl}$ vs. $v_{sg}$) where stratified flow is likely to occur (this is similar to what was done in the example for $\theta$ vs. $D$).

# Questions?

If you have any questions, feel free to send me an e-mail at `vik8290@utulsa.edu`.